In [1]:
import pandas as pd 
import numpy as np 
from zipfile import ZipFile
import os
import glob
import plotly.express as px

In [2]:
path = os.getcwd()
print(path)
print(os.listdir("prem/English Premier League (football) 10 Seasons/"))

C:\Users\mgxsl\github\PythonWork
['season-0910_csv.csv', 'season-1011_csv.csv', 'season-1112_csv.csv', 'season-1213_csv.csv', 'season-1314_csv.csv', 'season-1415_csv.csv', 'season-1516_csv.csv', 'season-1617_csv.csv', 'season-1718_csv.csv', 'season-1819_csv.csv']


In [3]:
glob.glob("*.zip")

['prem.zip']

In [4]:
### Read in from unzipped folder "prem"

file_list = list() 
path_to_csv = "prem/English Premier League (football) 10 Seasons/"
for file in os.listdir(path_to_csv):
#     print(path_to_csv + file)
    df=pd.read_csv(path_to_csv + file,sep=",")
    df['filename'] = file
    file_list.append(df)
    
all_szn = pd.concat(file_list, axis=0, ignore_index=True)
        

In [5]:
all_szn.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,filename,PSH,PSD,PSA,PSCH,PSCD,PSCA
0,E0,2009-08-15,Aston Villa,Wigan,0,2,A,0,1,A,...,1.22,4.40,3.99,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2009-08-15,Blackburn,Man City,0,2,A,0,1,A,...,2.38,1.60,1.54,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2009-08-15,Bolton,Sunderland,0,1,A,0,1,A,...,1.61,2.33,2.23,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2009-08-15,Chelsea,Hull,2,1,H,1,1,D,...,1.02,17.05,12.96,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2009-08-15,Everton,Arsenal,1,6,A,0,3,A,...,2.20,1.73,1.63,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## est. empty df
df_master = pd.DataFrame()

zip_files = glob.glob("*.zip")
#load all zip files in folder

## load the zip file first  - only one is read but i theory we could loop through several .zip
for zipfilename in zip_files:
#     print(zipfilename) 
    ## extract zip with ZipFile()
    zip_file = ZipFile(zipfilename)
    ## ZipFile.infolist() function is useful as it returns a list of the files zipped in the zipfolder, incl \
    ## individually call 
#     print(zip_file.infolist())
    
    for csv in zip_file.infolist():
        ## print full info of zip folder file
#         print(csv)
        ## print only the name 
#         print(csv.filename)
        ## read in the specific csv file in the zip folder. not wrap pandas read_csv around the ZipFile.open() function
        df = pd.read_csv(zip_file.open(csv.filename), delimiter=',',header=0)
        ## add col with specific file identifier 
        csv_name = str.split(csv.filename,"/")[1]
#         print(csv_name)
        df['filename'] = csv_name 
        ## concat the dfs from the zip folder
        df_master = pd.concat([df_master, df])

# print(df_master.info())

In [7]:
df_master.sample(10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,filename,PSH,PSD,PSA,PSCH,PSCD,PSCA
141,E0,02/12/17,Brighton,Liverpool,1,5,A,0,2,A,...,2.09,1.84,1.80,season-1718_csv.csv,7.58,4.55,1.50,7.25,4.39,1.53
97,E0,30/10/11,Tottenham,QPR,3,1,H,2,0,H,...,1.84,2.08,2.02,season-1112_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
144,E0,05/12/15,Stoke,Man City,2,0,H,2,0,H,...,1.97,1.94,1.89,season-1516_csv.csv,5.25,4.06,1.71,5.21,3.94,1.74
227,E0,22/01/11,Newcastle,Tottenham,1,1,D,0,0,D,...,2.46,1.60,1.53,season-1011_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
134,E0,29/11/17,Arsenal,Huddersfield,5,0,H,1,0,H,...,2.04,1.90,1.83,season-1718_csv.csv,1.23,7.23,14.43,1.18,8.30,18.50
298,E0,2010-03-20,Wigan,Burnley,1,0,H,0,0,D,...,1.30,3.75,3.27,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
80,E0,2009-10-17,Portsmouth,Tottenham,1,2,A,0,2,A,...,3.21,1.33,1.31,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
150,E0,08/12/2018,Arsenal,Huddersfield,1,0,H,0,0,D,...,1.80,2.12,2.06,season-1819_csv.csv,1.27,6.13,11.66,1.30,5.74,12.40
233,E0,26/01/11,Liverpool,Fulham,1,0,H,0,0,D,...,1.92,1.98,1.93,season-1011_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN
336,E0,2010-04-14,Wigan,Portsmouth,0,0,D,0,0,D,...,1.17,5.00,4.44,season-0910_csv.csv,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
display(df_master.dtypes.head(20),len(df_master),len(df_master.columns))

Div         object
Date        object
HomeTeam    object
AwayTeam    object
FTHG         int64
FTAG         int64
FTR         object
HTHG         int64
HTAG         int64
HTR         object
Referee     object
HS           int64
AS           int64
HST          int64
AST          int64
HF           int64
AF           int64
HC           int64
AC           int64
HY           int64
dtype: object

3800

78

In [9]:
mancity_home.sample(5)

print("number of matches:", len(df_master[df_master.HomeTeam == "Everton"]))

NameError: name 'mancity_home' is not defined

In [12]:
print("number of man city games:", len(df_master[df_master.HomeTeam == "Man City"]))

mancity_home = df_master[df_master.HomeTeam == "Man City"]
      
print(mancity_home["FTHG"].dtypes, mancity_home["FTAG"].dtypes)

mancity_home["Goal Diff"] = mancity_home["FTHG"] - mancity_home["FTAG"]

print("number of man city games w/ goal diff:", len(mancity_home["Goal Diff"]))
      
gp = mancity_home.groupby('Goal Diff').count()

display(gp.sample(4))

fig = px.bar(x = gp.index, y = gp.Date, color = gp.index,
             color_continuous_scale=px.colors.sequential.Cividis,
             labels = dict(y ="# of Games",
                           x ="Goal Difference/Game")
            )

fig.show()

## note - date aren't all the same 

number of man city games: 190
int64 int64
number of man city games w/ goal diff: 190


C:\Users\mgxsl\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,filename,PSH,PSD,PSA,PSCH,PSCD,PSCA
Goal Diff,,,,,,,,,,,,,,,,,,,,,
-3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,1,1,1,1,1,1
1,47,47,47,47,47,47,47,47,47,47,...,47,47,47,47,31,31,31,31,31,31
6,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,12,12,12,12,12,12


In [13]:

print(mancity_home.Date.head(5), mancity_home.Date.tail(5))
print('dates need fixing here')


19     2009-08-22
38     2009-09-12
65     2009-09-28
94     2009-10-25
109    2009-11-07
Name: Date, dtype: object 277    27/02/2019
293    09/03/2019
317    03/04/2019
338    20/04/2019
369    06/05/2019
Name: Date, dtype: object
dates need fixing here


### folium fun 


In [ ]:
import folium as f

### BeautifulSoup


In [14]:
import requests
from bs4 import BeautifulSoup 


In [20]:
response = requests.get("https://www.npr.org/2021/05/23/999599007/new-covid-19-cases-havent-been-this-low-in-the-u-s-since-last-june")
response.status_code


200

In [21]:
soup = BeautifulSoup(response.content, "html.parser")
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="noindex, nofollow" name="robots"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="utf-8" http-equiv="encoding"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, shrink-to-fit=no" name="viewport">
<title>NPR Cookie Consent and Choices</title>
<link href="https://s.npr.org/templates/css/fonts/Knockout.css" media="screen, print" rel="stylesheet"/>
<link href="https://s.npr.org/templates/css/fonts/GothamSSm.css" media="screen, print" rel="stylesheet"/>
<link href="css/choice-stylesheet.css" media="screen, print" rel="stylesheet"/>
<script src="./js/redirects.js" type="text/javascript"></script>
<script src="./js/domains.js" type="text/javascript"></script>
</meta></head>
<body>
<main class="content" id="content">
<header role="banner">
<img alt="NPR logo" class="npr-logo" src="https://media.npr.org/chrome_svg/npr-logo.svg"/>
<h1 class="header-txt">Cookie Consent

In [19]:
print(soup.title)
title = soup.title
print(title.string)

## find all p tags, that is paragraph
soup.find_all("p")


<title>NPR Cookie Consent and Choices</title>
NPR Cookie Consent and Choices


[<p>
             NPR’s sites use cookies, similar tracking and storage technologies, and information about the device you use to access our sites (together, “cookies”) to enhance your viewing, listening and user experience, personalize content, personalize messages from NPR’s sponsors, provide social media features, and analyze NPR’s traffic. This information is shared with social media, sponsorship, analytics, and other vendors or service providers. 
             <a href="https://text.npr.org/s.php?sId=609791368">See details</a>.
         </p>, <p>
             You may click on “<strong>Your Choices</strong>” below to learn about and use cookie management tools to limit use of cookies when you visit NPR’s sites. You can adjust your cookie choices in those tools at any time. If you click “<strong>Agree and Continue</strong>” below, you acknowledge that your cookie choices in those tools will be respected and that you otherwise agree to the use of cookies on NPR’s sites.
         </p>,

In [159]:
cal_table = soup.find("table")
cal_table_info = cal_table.find_all("td")
print(cal_table_info)
for date in cal_table_info:
    print(date.get_text())
    

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

The only difference is that `find_all()` returns a list containing the single result, and `find()` just returns the result.

If `find_all()` can’t find anything, it returns an empty list. If `find()` can’t find anything, it returns None: